### Imports

In [1]:
import torch
import torch.nn as nn
import torch_directml
import torchmetrics
import torch.optim as optim
import torchmetrics.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import os
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import ReduceLROnPlateau
import warnings
warnings.filterwarnings('ignore')
#AMD GPU Acceleration
device = torch_directml.device(torch_directml.default_device())

C:\Users\cafes\Envs\torch-dml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [3]:
set_seed()

In [4]:
train_writer = SummaryWriter('.\\runs\\train\\lstm\\')
val_writer = SummaryWriter('.\\runs\\validate\\lstm\\')

#### Read in train/test

In [5]:
train_df = pd.read_csv('train.csv', index_col='Time')
test_df = pd.read_csv('test.csv', index_col='Time')
train_df

,AccV,AccML,AccAP,Visit,Test,Medication,Age,Sex,YearsSinceDx,Event
Time,,,,,,,,,,
0,-0.418681,0.680295,-0.585967,-0.346080,0.570266,0.866530,-0.110647,-1.719166,-0.182777,0
1,-0.422574,0.678581,-0.579106,-0.346080,0.570266,0.866530,-0.110647,-1.719166,-0.182777,0
2,-0.421645,0.676684,-0.580472,-0.346080,0.570266,0.866530,-0.110647,-1.719166,-0.182777,0
3,-0.415656,0.674504,-0.594582,-0.346080,0.570266,0.866530,-0.110647,-1.719166,-0.182777,0
4,-0.405355,0.675046,-0.607290,-0.346080,0.570266,0.866530,-0.110647,-1.719166,-0.182777,0
...,...,...,...,...,...,...,...,...,...,...
136,-0.231824,0.202913,-1.195431,2.438148,-0.258162,-1.154029,-0.487734,0.581678,1.891825,0
137,-0.221527,0.198547,-1.207005,2.438148,-0.258162,-1.154029,-0.487734,0.581678,1.891825,0
138,-0.212499,0.204094,-1.211609,2.438148,-0.258162,-1.154029,-0.487734,0.581678,1.891825,0


In [6]:
test_df

,AccV,AccML,AccAP,Id,Visit,Test,Medication,Age,Sex,YearsSinceDx
Time,,,,,,,,,,
0,-0.420678,0.690095,-0.584365,003f117e14,-0.346080,0.570266,0.86653,-0.110647,-1.719166,-0.182777
1,-0.424043,0.687950,-0.593557,003f117e14,-0.346080,0.570266,0.86653,-0.110647,-1.719166,-0.182777
2,-0.413655,0.685621,-0.589727,003f117e14,-0.346080,0.570266,0.86653,-0.110647,-1.719166,-0.182777
3,-0.416550,0.688039,-0.585032,003f117e14,-0.346080,0.570266,0.86653,-0.110647,-1.719166,-0.182777
4,-0.431204,0.685708,-0.589775,003f117e14,-0.346080,0.570266,0.86653,-0.110647,-1.719166,-0.182777
...,...,...,...,...,...,...,...,...,...,...
281683,0.672098,-2.105837,1.189812,02ab235146,-0.509858,-1.086590,0.86653,1.272005,-1.719166,-0.342362
281684,0.632010,-2.109519,1.193029,02ab235146,-0.509858,-1.086590,0.86653,1.272005,-1.719166,-0.342362
281685,0.636255,-2.098329,1.198957,02ab235146,-0.509858,-1.086590,0.86653,1.272005,-1.719166,-0.342362


### Dataset & DataLoader

In [7]:
class ParkinsonDataset(Dataset):
    def __init__(self, df, lookback=15, goal='train'):
        super().__init__()
        self.data = df.to_numpy()
        self.lookback = lookback
        self.goal = goal

    # Return length of sample
    def __len__(self):
        return self.data.shape[0]

    # Get data at idx
    def __getitem__(self, idx):
        if self.goal == 'train':
            if idx < self.lookback:
                #If the index is less than lookback, pad with zeros
                features = torch.zeros(self.lookback, self.data.shape[1]-1, dtype=torch.float32)
                features[idx] = torch.tensor(self.data[idx, :-1], dtype=torch.float32)
                label = torch.tensor(self.data[idx, -1], dtype=torch.float32)
            else:
                #Otherwise, take the last lookback samples
                features = torch.tensor(self.data[idx - self.lookback:idx, :-1], dtype=torch.float32)
                label = torch.tensor(self.data[idx, -1], dtype=torch.float32)

            features = features.to(device)
            label = label.to(device)
            return features, label

        else:
            if idx < self.lookback:
                #If the index is less than lookback, pad with zeros
                features = torch.zeros(self.lookback, self.data.shape[1], dtype=torch.float32)
                features[idx] = torch.tensor(self.data[idx, :], dtype=torch.float32)
            else:
                #Otherwise, take the last lookback samples
                features = torch.tensor(self.data[idx - self.lookback:idx, :], dtype=torch.float32)

            features = features.to(device)
            return features

In [8]:
def train_val_split(df):
    #Split the training dataset into training/validation sets
    #80% Training - 20% Validation
    train_size = int(.8 * len(df))
    val_size = len(df)-train_size
    train_data, val_data = torch.utils.data.random_split(df, [train_size, val_size])

    #Instantiate DataLoaders
    train_loader = DataLoader(train_data,
                              batch_size=1024,
                              shuffle=False)

    val_loader = DataLoader(val_data,
                            batch_size=1024,
                            shuffle=False)
    
    return train_loader, val_loader

In [10]:
train = ParkinsonDataset(train_df, goal='train')
train_loader, val_loader = train_val_split(train)

In [11]:
test = test_df.drop(columns='Id')
test = ParkinsonDataset(test, goal='test')
test_loader = DataLoader(test, 
                         batch_size=1024, 
                         shuffle=False)

In [12]:
#accuracy metric
metric = torchmetrics.AveragePrecision(task='multiclass', num_classes=4, average='weighted', thresholds=4).to(device)

### Overfitting Function

In [13]:
#Function to determine whether the model is training appropriately
def overfit_batch(model, criterion, optimizer, batch, epochs=100):
    model = model.to(device)
    
    for epoch in range(epochs):
        metric.reset()
        loss_total = 0.0
        precision_total = 0.0
        for _ in range(len(batch)):
            features, label = batch
            features = features.to(device)
            label = label.to(device)
            
            optimizer.zero_grad()
            logits = model(features)
            loss = criterion(logits, label.long())
            ypred = torch.softmax(logits, dim=1)
            metric(ypred, label.long())
            precision=metric.compute()
            
            loss.backward()
            optimizer.step()

            loss_total += loss.item()
            precision_total += precision.item()

        avg_loss = loss_total / len(batch)
        avg_precision = precision_total / len(batch)
        if epoch %10 == 9:
            print(f'Epoch [{epoch + 1}/{epochs}] | Avg Loss: {avg_loss:.4f} | Avg Precision: {(avg_precision * 100.0):.2f}')

### Train | Validate | Predict

In [14]:
def predict(model, dataloader):
    predictions = []
    with tqdm(dataloader, colour='yellow') as pbar:
        for features in pbar:
            features = features.to(device)
            preds = model(features)
            preds = torch.argmax(preds, dim=1)
            predictions.extend(preds.cpu().numpy())
    
    return predictions

In [15]:
#Validate one epoch
def validate(model, criterion, dataloader, epoch):
    model.eval()
    metric.reset()
    
    loss_total = 0.0
    precision_total = 0.0
    
    with tqdm(dataloader, desc='Validate', total=len(dataloader), unit='batch', colour='blue') as pbar:
        with torch.no_grad():
            for batch in pbar:
                features, label = batch
                features = features.to(device)
                label = label.to(device)

                logits = model(features)               
                #Update running loss and precision
                loss = criterion(logits, label.long())
                ypred = torch.softmax(logits, dim=1)
                metric(ypred, label.long())
                precision=metric.compute()
                
                precision_total += precision.item()
                loss_total += loss.item()

                #Update the tqdm progress bar description with loss and precision
                pbar.set_postfix_str(s=f'Loss: {loss:.4f} | Precision: {precision*100:.2f}%')
                
        avg_loss = loss_total / len(dataloader)
        avg_precision = precision_total / len(dataloader)
        
    model.train()    
    return avg_loss, avg_precision

In [16]:
def train_validate(model, criterion, optimizer, scheduler, epochs=100, train_loader=train_loader, val_loader=val_loader, patience=5):
    model = model.train()
    best_precision = None
    min_delta = 0
    counter = 0
    
    for epoch in range(epochs):
        metric.reset()
        loss_total = 0.0
        precision_total = 0.0
        with tqdm(train_loader, desc='Train', total=len(train_loader), unit='batch', colour='green') as pbar:
            for batch in pbar:
                #Set features/targets to device
                features, label = batch
                features = features.to(device)
                label = label.to(device)

                #Predict labels
                optimizer.zero_grad()                
                logits = model(features)
                loss = criterion(logits, label.long())
                ypred = torch.softmax(logits, dim=1)
                metric(ypred, label.long())
                precision=metric.compute()
                
                #Tensorboard
                train_writer.add_scalar('Train-Loss', loss, epoch)
                train_writer.add_scalar('Train-Precision', precision, epoch)
                
                pbar.set_postfix_str(s=f'Loss: {loss:.4f} | Precision: {precision*100:.2f}%')

                #Adjust gradient
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
                optimizer.step()

                #Accumulate total loss and precision
                loss_total += loss.item()
                precision_total += precision.item()

            avg_loss = loss_total / len(train_loader)
            avg_precision = precision_total / len(train_loader)
         
        #Report training loss/precision
        print(f'Epoch {epoch + 1} | Avg Loss: {avg_loss:.4f} | Avg Precision: {(avg_precision * 100.0):.2f}%')

        if epoch%3==2:
            #Validate one epoch and update lr scheduler
            avg_val_loss, avg_val_precision = validate(model, criterion, val_loader, epoch)
            scheduler.step(avg_val_precision)
        
            #Early Stopping
            if best_precision == None:
                best_precision = avg_val_precision
            elif(best_precision - avg_val_precision < min_delta):
                best_precision = avg_val_precision
                counter = 0
            elif(best_precision - avg_val_precision > min_delta):
                counter += 1
                if(counter >= patience):
                    print(f'Early Stopping: Precision stagnated or decreased too many times. Best Validation: {(best_precision*100.0):.2f}%')
                    break

            print(f'Epoch {epoch + 1} | Avg Loss: {avg_val_loss:.4f} | Avg Precision: {(avg_val_precision * 100.0):.2f}% | Patience: {counter}')

    train_writer.flush()
    train_writer.close()
    val_writer.flush()
    val_writer.close()
    print(f'Training Complete - Best Validate Precision: {(best_precision*100.0):.2f}%')
    
    return model

## Model Architecture

In [17]:
class LSTMnet(nn.Module):
    def __init__(self, num_inputs, hidden_size, num_layers, num_outputs):
        super(LSTMnet, self).__init__()
        self.hidden = hidden_size
        self.layers = num_layers
        self.lstm = nn.LSTM(num_inputs,
                            hidden_size,
                            num_layers,
                            batch_first=True)

        for name, param in self.lstm.named_parameters():
            if 'weight' in name:
                nn.init.kaiming_normal_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0.0)
    
        self.batch_norm1 = nn.BatchNorm1d(hidden_size)
        self.elu = nn.ELU()
        self.fc1 = nn.Linear(hidden_size, num_outputs)
        nn.init.xavier_uniform_(self.fc1.weight)

    def forward(self, x):
        c0 = torch.zeros((self.layers, x.size(0), self.hidden), dtype=torch.float32).to(device)
        h0 = torch.zeros((self.layers, x.size(0), self.hidden), dtype=torch.float32).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = out[:, -1, :]

        # Apply batch normalization
        out = self.batch_norm1(out)
        out = self.elu(out)
        out = self.fc1(out) 
        return out

### Overfitting

In [65]:
batch = next(iter(train_loader))
model = LSTMnet(9, 500, 1, 4)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.01)


overfit_batch(model, criterion, optimizer, batch)

Epoch [10/100] | Avg Loss: 0.3972 | Avg Precision: 79.00
Epoch [20/100] | Avg Loss: 0.2422 | Avg Precision: 89.05
Epoch [30/100] | Avg Loss: 0.1192 | Avg Precision: 96.13
Epoch [40/100] | Avg Loss: 0.0619 | Avg Precision: 98.40
Epoch [50/100] | Avg Loss: 0.0456 | Avg Precision: 98.75
Epoch [60/100] | Avg Loss: 0.0260 | Avg Precision: 99.76
Epoch [70/100] | Avg Loss: 0.0088 | Avg Precision: 100.00
Epoch [80/100] | Avg Loss: 0.0041 | Avg Precision: 100.00
Epoch [90/100] | Avg Loss: 0.0020 | Avg Precision: 100.00
Epoch [100/100] | Avg Loss: 0.0011 | Avg Precision: 100.00


# Training

## Downsample

### 11-1-23

#### Model 5 - Batch 1024 - Lookback 7 - Scheduler patience update

In [18]:
model = LSTMnet(9, 150, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.03)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=1)
epochs = 100

model5 = train_validate(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)
torch.save(model5.state_dict(), '.\\models\\downsample\\lstm\\model5.pth')

Train:  21%|█████▎                   | 63/295 [00:12<00:47,  4.85batch/s, Loss: 0.4164 | Precision: 76.91%]


KeyboardInterrupt: 

### 11-3-23

#### Model 6 - Scheduler adjustment; using subset df

In [49]:
model = LSTMnet(9, 150, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.02)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.8, patience=2, min_lr=.001)
epochs = 10

model6 = train_validate(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)
torch.save(model6.state_dict(), '.\\models\\subset\\lstm\\model6.pth')

Train: 100%|██████████████████████| 3560/3560 [09:54<00:00,  5.99batch/s, Loss: 0.4148 | Precision: 79.91%]


Epoch 1 | Avg Loss: 0.4096 | Avg Precision: 77.97%


Train: 100%|██████████████████████| 3560/3560 [09:53<00:00,  6.00batch/s, Loss: 0.3906 | Precision: 82.25%]


Epoch 2 | Avg Loss: 0.3711 | Avg Precision: 82.08%


Train: 100%|██████████████████████| 3560/3560 [09:54<00:00,  5.99batch/s, Loss: 0.3779 | Precision: 82.66%]


...

Epoch 9 | Avg Loss: 0.3701 | Avg Precision: 82.39%


Validate: 100%|█████████████████████| 890/890 [02:20<00:00,  6.32batch/s, Loss: 0.3699 | Precision: 82.73%]


Epoch 9 | Avg Loss: 0.3697 | Avg Precision: 82.72% | Patience: 2


Train: 100%|██████████████████████| 3560/3560 [09:52<00:00,  6.01batch/s, Loss: 0.3853 | Precision: 82.64%]

Epoch 10 | Avg Loss: 0.3669 | Avg Precision: 82.54%
Training Complete


#### Model 7 - Downsample

In [58]:
model = LSTMnet(9, 150, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.02)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001)
epochs = 100

model7 = train_validate(model, criterion, optimizer, scheduler, epochs, train_loader, val_loader)
torch.save(model7.state_dict(), '.\\models\\downsample\\lstm\\model7.pth')

Train: 100%|████████████████████████| 295/295 [00:48<00:00,  6.03batch/s, Loss: 0.2805 | Precision: 74.84%]


Epoch 1 | Avg Loss: 0.4961 | Avg Precision: 72.52%


Train: 100%|████████████████████████| 295/295 [00:48<00:00,  6.06batch/s, Loss: 0.2102 | Precision: 78.18%]


Epoch 2 | Avg Loss: 0.4336 | Avg Precision: 77.37%


Train: 100%|████████████████████████| 295/295 [00:48<00:00,  6.03batch/s, Loss: 0.1772 | Precision: 79.39%]


...

Epoch 71 | Avg Loss: 0.3557 | Avg Precision: 83.22%


Train: 100%|████████████████████████| 295/295 [00:48<00:00,  6.08batch/s, Loss: 0.2009 | Precision: 82.79%]


Epoch 72 | Avg Loss: 0.3587 | Avg Precision: 82.86%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.45batch/s, Loss: 0.3876 | Precision: 83.29%]

Early Stopping: Precision stagnated or decreased too many times. Best Validation: 83.64%
Training Complete


#### Model 8 - Lookback 12 - Batch 250

In [74]:
model = LSTMnet(9, 150, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.02)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=2, min_lr=.001)
epochs = 100

model8 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model8.state_dict(), '.\\models\\downsample\\lstm\\model8.pth')

Train: 100%|████████████████████████| 402/402 [00:57<00:00,  7.01batch/s, Loss: 0.4796 | Precision: 76.39%]


Epoch 1 | Avg Loss: 0.4749 | Avg Precision: 73.46%


Train: 100%|████████████████████████| 402/402 [00:57<00:00,  6.95batch/s, Loss: 0.4691 | Precision: 79.57%]


Epoch 2 | Avg Loss: 0.4128 | Avg Precision: 79.26%


Train: 100%|████████████████████████| 402/402 [00:56<00:00,  7.13batch/s, Loss: 0.4669 | Precision: 80.07%]


...

Epoch 20 | Avg Loss: 0.4321 | Avg Precision: 78.13%


Train: 100%|████████████████████████| 402/402 [00:56<00:00,  7.09batch/s, Loss: 0.4254 | Precision: 78.42%]


Epoch 21 | Avg Loss: 0.4303 | Avg Precision: 78.33%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.34batch/s, Loss: 0.4107 | Precision: 78.43%]

Early Stopping: Precision stagnated or decreased too many times. Best Validation: 81.23%
Training Complete


#### Model 9 - Lookback 6 - Xavier weight - 250 Nodes

In [35]:
model = LSTMnet(9, 250, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.02)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001)

model9 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model9.state_dict(), '.\\models\\downsample\\lstm\\model9.pth')

Train: 100%|████████████████████████| 295/295 [01:03<00:00,  4.66batch/s, Loss: 0.3358 | Precision: 74.68%]


Epoch 1 | Avg Loss: 0.4986 | Avg Precision: 71.97%


Train: 100%|████████████████████████| 295/295 [01:03<00:00,  4.66batch/s, Loss: 0.3156 | Precision: 78.25%]


Epoch 2 | Avg Loss: 0.4316 | Avg Precision: 77.67%


Train: 100%|████████████████████████| 295/295 [01:03<00:00,  4.67batch/s, Loss: 0.3209 | Precision: 79.49%]


...

Epoch 32 | Avg Loss: 0.3776 | Avg Precision: 81.64%


Train: 100%|████████████████████████| 295/295 [01:03<00:00,  4.67batch/s, Loss: 0.2346 | Precision: 81.95%]


Epoch 33 | Avg Loss: 0.3803 | Avg Precision: 81.80%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  5.94batch/s, Loss: 0.4143 | Precision: 81.11%]

Early Stopping: Precision stagnated or decreased too many times. Best Validation: 81.73%
Training Complete


### 11-5-23

#### Model 10 - Lookback 30 - 100 Nodes

In [22]:
model = LSTMnet(9, 100, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.02)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001)
model10 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model10.state_dict(), '.\\models\\downsample\\lstm\\model10.pth')

Train: 100%|████████████████████████| 295/295 [01:05<00:00,  4.53batch/s, Loss: 0.3978 | Precision: 74.85%]


Epoch 1 | Avg Loss: 0.4903 | Avg Precision: 71.89%


Train: 100%|████████████████████████| 295/295 [01:05<00:00,  4.52batch/s, Loss: 0.4019 | Precision: 79.10%]


Epoch 2 | Avg Loss: 0.4194 | Avg Precision: 78.60%


Validate: 100%|███████████████████████| 74/74 [00:13<00:00,  5.63batch/s, Loss: 0.5121 | Precision: 78.05%]


...

Epoch 35 | Avg Loss: 0.3646 | Avg Precision: 82.86%


Train: 100%|████████████████████████| 295/295 [01:05<00:00,  4.49batch/s, Loss: 0.3947 | Precision: 83.24%]


Epoch 36 | Avg Loss: 0.3598 | Avg Precision: 83.28%


Validate: 100%|███████████████████████| 74/74 [00:13<00:00,  5.57batch/s, Loss: 0.4456 | Precision: 81.73%]

Early Stopping: Precision stagnated or decreased too many times. Best Validation: 83.59%
Training Complete - Best Precision: 83.59%


#### Model 11 - Lookback 30 - LR adjust

In [24]:
model = LSTMnet(9, 100, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001)

model11 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model11.state_dict(), '.\\models\\downsample\\lstm\\model11.pth')

Train: 100%|████████████████████████| 295/295 [01:05<00:00,  4.47batch/s, Loss: 0.3680 | Precision: 75.71%]


Epoch 1 | Avg Loss: 0.4804 | Avg Precision: 72.28%


Train: 100%|████████████████████████| 295/295 [01:05<00:00,  4.48batch/s, Loss: 0.3682 | Precision: 80.72%]


Epoch 2 | Avg Loss: 0.3918 | Avg Precision: 79.95%


Validate: 100%|███████████████████████| 74/74 [00:13<00:00,  5.51batch/s, Loss: 0.4930 | Precision: 76.42%]


...

Epoch 97 | Avg Loss: 0.2255 | Avg Precision: 90.02%


Train: 100%|████████████████████████| 295/295 [01:04<00:00,  4.60batch/s, Loss: 0.0434 | Precision: 89.95%]


Epoch 98 | Avg Loss: 0.2266 | Avg Precision: 89.98%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  5.74batch/s, Loss: 0.2104 | Precision: 89.12%]

Early Stopping: Precision stagnated or decreased too many times. Best Validation: 89.76%
Training Complete - Best Precision: 89.76%


#### Model 12 - Lookback 20

In [29]:
model = LSTMnet(9, 100, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001)

model12 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model12.state_dict(), '.\\models\\downsample\\lstm\\model12.pth')

Train: 100%|████████████████████████| 295/295 [01:00<00:00,  4.89batch/s, Loss: 0.8911 | Precision: 75.16%]


Epoch 1 | Avg Loss: 0.4923 | Avg Precision: 72.27%


Train: 100%|████████████████████████| 295/295 [01:00<00:00,  4.87batch/s, Loss: 0.7949 | Precision: 79.80%]


Epoch 2 | Avg Loss: 0.4075 | Avg Precision: 79.31%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  5.96batch/s, Loss: 0.5814 | Precision: 69.64%]


...

Epoch 99 | Avg Loss: 0.2702 | Avg Precision: 87.99%


Train: 100%|████████████████████████| 295/295 [01:00<00:00,  4.86batch/s, Loss: 0.4216 | Precision: 87.89%]


Epoch 100 | Avg Loss: 0.2695 | Avg Precision: 88.00%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  5.95batch/s, Loss: 0.2397 | Precision: 87.86%]

Epoch 100 | Avg Loss: 0.2822 | Avg Precision: 87.83% | Patience: 2
Training Complete - Best Validate Precision: 87.94%


### 11-6-23

#### Model 13 - Lookback 10

In [16]:
model = LSTMnet(9, 100, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001)

model13 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model13.state_dict(), '.\\models\\downsample\\lstm\\model13.pth')

Train: 100%|████████████████████████| 295/295 [00:57<00:00,  5.15batch/s, Loss: 0.3792 | Precision: 75.37%]


Epoch 1 | Avg Loss: 0.4884 | Avg Precision: 72.10%


Train: 100%|████████████████████████| 295/295 [00:56<00:00,  5.25batch/s, Loss: 0.4054 | Precision: 80.39%]


Epoch 2 | Avg Loss: 0.3990 | Avg Precision: 79.71%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.21batch/s, Loss: 0.4766 | Precision: 78.38%]


...

Epoch 99 | Avg Loss: 0.2414 | Avg Precision: 89.03%


Train: 100%|████████████████████████| 295/295 [00:55<00:00,  5.30batch/s, Loss: 0.1629 | Precision: 89.03%]


Epoch 100 | Avg Loss: 0.2416 | Avg Precision: 89.02%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.23batch/s, Loss: 0.2367 | Precision: 87.97%]

Epoch 100 | Avg Loss: 0.2593 | Avg Precision: 87.79% | Patience: 6
Training Complete - Best Validate Precision: 88.30%


#### Model 14 - Lookback 7

In [29]:
model = LSTMnet(9, 100, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001)

model14 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model14.state_dict(), '.\\models\\downsample\\lstm\\model14.pth')

Train: 100%|████████████████████████| 295/295 [00:53<00:00,  5.49batch/s, Loss: 0.4268 | Precision: 75.01%]


Epoch 1 | Avg Loss: 0.4959 | Avg Precision: 71.94%


Train: 100%|████████████████████████| 295/295 [00:53<00:00,  5.48batch/s, Loss: 0.3364 | Precision: 79.71%]


Epoch 2 | Avg Loss: 0.4078 | Avg Precision: 79.03%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.56batch/s, Loss: 0.4525 | Precision: 78.67%]


...

Epoch 99 | Avg Loss: 0.2672 | Avg Precision: 87.79%


Train: 100%|████████████████████████| 295/295 [00:53<00:00,  5.50batch/s, Loss: 0.1361 | Precision: 87.84%]


Epoch 100 | Avg Loss: 0.2670 | Avg Precision: 87.83%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.42batch/s, Loss: 0.3025 | Precision: 87.67%]

Epoch 100 | Avg Loss: 0.2867 | Avg Precision: 87.65% | Patience: 0
Training Complete - Best Validate Precision: 87.65%


#### Model 15 - Lookback 30 - Batch 512

In [36]:
model = LSTMnet(9, 100, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.001)

model15 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model15.state_dict(), '.\\models\\downsample\\lstm\\model15.pth')

Train: 100%|████████████████████████| 295/295 [00:54<00:00,  5.45batch/s, Loss: 0.4262 | Precision: 74.88%]


Epoch 1 | Avg Loss: 0.4942 | Avg Precision: 71.89%


Train: 100%|████████████████████████| 295/295 [00:54<00:00,  5.46batch/s, Loss: 0.2509 | Precision: 80.30%]


Epoch 2 | Avg Loss: 0.3978 | Avg Precision: 79.37%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.41batch/s, Loss: 0.4953 | Precision: 75.67%]


...

Epoch 99 | Avg Loss: 0.2661 | Avg Precision: 87.70%


Train: 100%|████████████████████████| 295/295 [00:53<00:00,  5.49batch/s, Loss: 0.0954 | Precision: 87.67%]


Epoch 100 | Avg Loss: 0.2660 | Avg Precision: 87.69%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.32batch/s, Loss: 0.3055 | Precision: 87.27%]

Epoch 100 | Avg Loss: 0.2845 | Avg Precision: 87.29% | Patience: 0
Training Complete - Best Validate Precision: 87.29%


#### Model 16 - Batch 1024 - Lookback 30 - 200 Nodes

In [42]:
model = LSTMnet(9, 75, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=2, min_lr=.001)

model16 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model16.state_dict(), '.\\models\\downsample\\lstm\\model16.pth')

Train: 100%|████████████████████████| 295/295 [00:57<00:00,  5.10batch/s, Loss: 0.3557 | Precision: 74.88%]


Epoch 1 | Avg Loss: 0.4924 | Avg Precision: 71.91%


Train: 100%|████████████████████████| 295/295 [00:58<00:00,  5.06batch/s, Loss: 0.2975 | Precision: 79.41%]


Epoch 2 | Avg Loss: 0.4143 | Avg Precision: 78.62%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  6.05batch/s, Loss: 0.4747 | Precision: 75.07%]


...

Epoch 58 | Avg Loss: 0.2929 | Avg Precision: 86.39%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  5.83batch/s, Loss: 0.3147 | Precision: 85.49%]


Epoch 58 | Avg Loss: 0.3242 | Avg Precision: 85.41% | Patience: 6


Train:  38%|█████████               | 112/295 [00:22<00:36,  4.95batch/s, Loss: 0.3185 | Precision: 86.36%]


KeyboardInterrupt: 

### 11-9-23

#### Model 17 - Lookback 15 - Validate Every 3

In [19]:
model = LSTMnet(9, 175, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=2, min_lr=.001)

model17 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model17.state_dict(), '.\\models\\downsample\\lstm\\model17.pth')

Train: 100%|████████████████████████| 295/295 [00:55<00:00,  5.32batch/s, Loss: 0.4122 | Precision: 75.75%]


Epoch 1 | Avg Loss: 0.4832 | Avg Precision: 72.38%


Train: 100%|████████████████████████| 295/295 [00:55<00:00,  5.29batch/s, Loss: 0.3309 | Precision: 80.91%]


Epoch 2 | Avg Loss: 0.3885 | Avg Precision: 80.09%


Train: 100%|████████████████████████| 295/295 [00:55<00:00,  5.29batch/s, Loss: 0.3512 | Precision: 82.59%]


...

Epoch 99 | Avg Loss: 0.2497 | Avg Precision: 88.46%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.35batch/s, Loss: 0.2501 | Precision: 88.40%]


Epoch 99 | Avg Loss: 0.2554 | Avg Precision: 88.32% | Patience: 0


Train: 100%|████████████████████████| 295/295 [00:57<00:00,  5.14batch/s, Loss: 0.1129 | Precision: 88.60%]

Epoch 100 | Avg Loss: 0.2491 | Avg Precision: 88.55%
Training Complete - Best Validate Precision: 88.32%


#### Model 18 - Batch 1024 - Validate Every 5

In [18]:
model = LSTMnet(9, 100, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=2, min_lr=.001)
epochs = 200

model18 = train_validate(model, criterion, optimizer, scheduler, epochs)
torch.save(model18.state_dict(), '.\\models\\downsample\\lstm\\model18.pth')

Train: 100%|████████████████████████| 295/295 [00:57<00:00,  5.16batch/s, Loss: 0.3489 | Precision: 75.30%]


Epoch 1 | Avg Loss: 0.4909 | Avg Precision: 71.88%


Train: 100%|████████████████████████| 295/295 [00:57<00:00,  5.14batch/s, Loss: 0.3910 | Precision: 80.66%]


Epoch 2 | Avg Loss: 0.3950 | Avg Precision: 79.93%


Train: 100%|████████████████████████| 295/295 [00:57<00:00,  5.09batch/s, Loss: 0.3246 | Precision: 82.08%]


...

Epoch 69 | Avg Loss: 0.2768 | Avg Precision: 87.17%


Train: 100%|████████████████████████| 295/295 [00:58<00:00,  5.03batch/s, Loss: 0.1261 | Precision: 87.14%]


Epoch 70 | Avg Loss: 0.2779 | Avg Precision: 87.08%


Validate: 100%|███████████████████████| 74/74 [00:11<00:00,  6.20batch/s, Loss: 0.2716 | Precision: 86.93%]

Early Stopping: Precision stagnated or decreased too many times. Best Validation: 87.16%
Training Complete - Best Validate Precision: 87.16%


### 11-11-23

#### Model 1 - Batch 1024 - Changed weight initializations

In [20]:
model = LSTMnet(9, 300, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.0001, verbose=True)
epochs = 200

model1 = train_validate(model, criterion, optimizer, scheduler, epochs)
torch.save(model1.state_dict(), '.\\models\\downsample\\lstm\\model1.pth')

Train: 100%|████████████████████████| 295/295 [01:02<00:00,  4.73batch/s, Loss: 0.3102 | Precision: 76.60%]


Epoch 1 | Avg Loss: 0.4660 | Avg Precision: 73.22%


Train: 100%|████████████████████████| 295/295 [01:02<00:00,  4.72batch/s, Loss: 0.3082 | Precision: 81.21%]


Epoch 2 | Avg Loss: 0.3825 | Avg Precision: 80.57%


Train: 100%|████████████████████████| 295/295 [01:02<00:00,  4.71batch/s, Loss: 0.2522 | Precision: 82.59%]


...

Epoch 119 | Avg Loss: 0.2450 | Avg Precision: 88.74%


Train: 100%|████████████████████████| 295/295 [01:02<00:00,  4.75batch/s, Loss: 0.0629 | Precision: 88.81%]


Epoch 120 | Avg Loss: 0.2455 | Avg Precision: 88.75%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  5.83batch/s, Loss: 0.2670 | Precision: 88.13%]

Epoch 00030: reducing learning rate of group 0 to 4.7830e-03.
Early Stopping: Precision stagnated or decreased too many times. Best Validation: 88.51%
Training Complete - Best Validate Precision: 88.51%


### 11-12-23

#### Model 2 - Batch 1024 - LB 15 - 500 Units

In [28]:
model = LSTMnet(9, 500, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.0001, verbose=True)
epochs = 200

model2 = train_validate(model, criterion, optimizer, scheduler, epochs)
torch.save(model2.state_dict(), '.\\models\\downsample\\lstm\\model2.pth')

Train: 100%|████████████████████████| 295/295 [01:09<00:00,  4.21batch/s, Loss: 0.4697 | Precision: 77.23%]


Epoch 1 | Avg Loss: 0.4533 | Avg Precision: 74.01%


Train: 100%|████████████████████████| 295/295 [01:07<00:00,  4.40batch/s, Loss: 0.4399 | Precision: 81.23%]


Epoch 2 | Avg Loss: 0.3820 | Avg Precision: 80.86%


Train: 100%|████████████████████████| 295/295 [01:07<00:00,  4.40batch/s, Loss: 0.3444 | Precision: 82.26%]


...

Epoch 199 | Avg Loss: 0.2476 | Avg Precision: 88.92%


Train: 100%|████████████████████████| 295/295 [01:05<00:00,  4.48batch/s, Loss: 0.1745 | Precision: 88.83%]


Epoch 200 | Avg Loss: 0.2468 | Avg Precision: 88.92%


Validate: 100%|███████████████████████| 74/74 [00:13<00:00,  5.49batch/s, Loss: 0.3003 | Precision: 88.38%]

Epoch 200 | Avg Loss: 0.2619 | Avg Precision: 88.36% | Patience: 1
Training Complete - Best Validate Precision: 88.46%


### 11-13-23

#### Model 3 - Same as 11-12-23 using new downsample features

In [15]:
model = LSTMnet(8, 500, 1, 4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=.01)
scheduler = ReduceLROnPlateau(optimizer, 'max', factor=.9, patience=1, min_lr=.0001, verbose=True)

model3 = train_validate(model, criterion, optimizer, scheduler)
torch.save(model3.state_dict(), '.\\models\\downsample\\lstm\\model3.pth')

Train: 100%|████████████████████████| 295/295 [01:04<00:00,  4.57batch/s, Loss: 0.2697 | Precision: 82.22%]


Epoch 1 | Avg Loss: 0.3756 | Avg Precision: 79.01%


Train: 100%|████████████████████████| 295/295 [01:03<00:00,  4.63batch/s, Loss: 0.0645 | Precision: 86.76%]


Epoch 2 | Avg Loss: 0.2791 | Avg Precision: 85.86%


Train: 100%|████████████████████████| 295/295 [01:04<00:00,  4.58batch/s, Loss: 0.0335 | Precision: 88.83%]


...

Epoch 99 | Avg Loss: 0.0345 | Avg Precision: 98.82%


Train: 100%|████████████████████████| 295/295 [01:02<00:00,  4.70batch/s, Loss: 0.0042 | Precision: 98.84%]


Epoch 100 | Avg Loss: 0.0340 | Avg Precision: 98.85%


Validate: 100%|███████████████████████| 74/74 [00:12<00:00,  5.82batch/s, Loss: 0.1695 | Precision: 96.23%]

Epoch 100 | Avg Loss: 0.1259 | Avg Precision: 96.21% | Patience: 1
Training Complete - Best Validate Precision: 96.77%


## Predict | Submit

In [ ]:
ypreds = predict(model1, test_loader)
test_df['Prediction'] = ypreds

In [ ]:
sub = pd.DataFrame()
temp = test_df.copy()
temp["Id"] = temp.apply(lambda x : str(x.Id) + "_" + str(x.Time), axis = 1)
temp['StartHesitation'] = np.where(temp['Prediction'] == 1, 1, 0)
temp['Turn'] = np.where(temp['Prediction'] == 2, 1, 0)
temp['Walking'] = np.where(temp['Prediction'] == 3, 1, 0)
temp = temp[['Id', 'StartHesitation', 'Turn', 'Walking']]
sub = pd.concat([sub, temp])